# 第5章 数据类构建器

## 5.1 数据类构建器概述

1. 直接通过类构建

In [1]:
# 表示地理位置的经纬度
class Coordinate:
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon

In [3]:
moscow = Coordinate(55.76, 37.62)
location = Coordinate(55.76, 37.62)
location == moscow

False

In [4]:
(location.lat, location.lon) == (moscow.lat, moscow.lon)

True

2. 使用namedtuple构建Coordinate类

In [7]:
from collections import namedtuple

Coordinate = namedtuple('Coordinate', 'lat lon')
issubclass(Coordinate, tuple)

True

In [6]:
moscow = Coordinate(55.76, 37.62)
moscow == Coordinate(lat=55.76, lon=37.62)

True

3. 使用typing.NamedTuple构建

In [8]:
from typing import NamedTuple

class Coordinate(NamedTuple):
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

4. 使用dataclass装饰器声明实例属性

In [9]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

## 5.2 典型的具名元组

1. 定义并使用一个具名元组类型

In [15]:
from collections import namedtuple

City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [11]:
tokyo.population, tokyo.coordinates, tokyo[1]

(36.933, (35.689722, 139.691667), 'JP')

2. 具名元组的属性和方法

In [16]:
City._fields

('name', 'country', 'population', 'coordinates')

In [17]:
Coordinate = namedtuple('Coordinate', 'lat lon')
delhi_data = ('Delhi NCR', 'IN', 21.935, Coordinate(28.613889, 77.208889))
delhi = City._make(delhi_data)
delhi._asdict()

{'name': 'Delhi NCR',
 'country': 'IN',
 'population': 21.935,
 'coordinates': Coordinate(lat=28.613889, lon=77.208889)}

In [18]:
import json

json.dumps(delhi._asdict())

'{"name": "Delhi NCR", "country": "IN", "population": 21.935, "coordinates": [28.613889, 77.208889]}'

3. 构建一个具名元组，为字段指定默认值

In [20]:
Coordinate = namedtuple('Coordinate', 'lat lon reference', defaults=['WSG84']) 
Coordinate(0, 0)

Coordinate(lat=0, lon=0, reference='WSG84')

In [21]:
Coordinate._field_defaults

{'reference': 'WSG84'}

## 5.3 @dataclass详解

1. @dataclass定义

@dataclass(*, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)

- init：默认值是True，生成\_\_init\_\_，如果用户自己实现了\_\_init\_\_，则忽略该参数。
- repr：默认值是True，生成\_\_repr\_\_，如果用户自己实现了\_\_repr\_\_，则忽略该参数。
- eq：默认值是True，生成\_\_eq\_\_，如果用户自己实现了\_\_eq\_\_，则忽略该参数。
- order：默认值是False，生成\_\_lt\_\_、\_\_le\_\_、\_\_gt\_\_、\_\_ge\_\_、如果eq=False，或者执行定义或继承其他用于比较的方法，则抛出异常。
- unsafe_hase：默认值是False，生成\_\_hash\_\_。
- frozen：默认值是False，让实例不可变。

In [22]:
# 定义ClubMember
from dataclasses import dataclass, field

@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)    

2. 都柏林核心模式

都柏林核心模式是一小组术语，可用于描述数字资源（视频、图像、网页等），也可用于描述物理资源，例如图书、CD和艺术品等对象。

In [29]:
from dataclasses import dataclass, field, fields
from typing import Optional
from enum import Enum, auto
from datetime import date


class ResourceType(Enum):  # <1>
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()


@dataclass
class Resource:
    """描述媒体资源"""
    identifier: str                                   
    title: str = '<untitled>'                          
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None                        
    type: ResourceType = ResourceType.BOOK             
    description: str = ''
    language: str = ''
    subjects: list[str] = field(default_factory=list)
        
    def __repr__(self):
        cls = self.__class__
        cls_name = cls.__name__
        indent = ' ' * 4
        res = [f'{cls_name}(']                            
        for f in fields(cls):                             
            value = getattr(self, f.name)                 
            res.append(f'{indent}{f.name} = {value!r},')  

        res.append(')')                                   
        return '\n'.join(res)                             

In [30]:
description = 'Improving the design of existing code'
book = Resource('978-0-13-475759-9', 'Refactoring, 2nd Edition', 
                ['Martin Fowler', 'Kent Beck'], date(2018, 11, 19),
                ResourceType.BOOK, description, 'EN',
                ['computer programming', 'OOP'])

In [31]:
book

Resource(
    identifier = '978-0-13-475759-9',
    title = 'Refactoring, 2nd Edition',
    creators = ['Martin Fowler', 'Kent Beck'],
    date = datetime.date(2018, 11, 19),
    type = <ResourceType.BOOK: 1>,
    description = 'Improving the design of existing code',
    language = 'EN',
    subjects = ['computer programming', 'OOP'],
)

## 5.4 杂谈

- Python一直都有声明类属性的简便方式，实例属性更常用，直接在`__init__`方法中创建实例属性。
- @dataclass解决了上述问题，类型提示始终是可选的。